<a href="https://colab.research.google.com/github/anaghasid/GenPruning/blob/main/tweet_topic_Pruning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install --quiet datasets
!pip install --quiet evaluate
!pip install --quiet accelerate
!pip install --quiet bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.3/474.3 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 9.5 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 17.0.0 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.4/122.4 MB 7.7 MB/s eta 0:00:00


In [ ]:
import numpy as np
import torch
import evaluate
from transformers import AutoModelForSequenceClassification, AutoTokenizer, TrainingArguments, Trainer, BitsAndBytesConfig
from datasets import load_dataset
import accelerate
from sklearn.metrics import accuracy_score
import time
from torch.utils.data import DataLoader
from collections import defaultdict
import pandas as pd
import random

In [ ]:
model_name = "cardiffnlp/tweet-topic-latest-multi"

In [ ]:
def make_model():
  device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
  model = AutoModelForSequenceClassification.from_pretrained(model_name, trust_remote_code=True,
                                                           device_map=device)
  return model

In [ ]:
# Load a pretrained model and tokenizer
quantization_config = BitsAndBytesConfig(
    load_in_4bit = True, # enable 4-bit quantization
    bnb_4bit_quant_type = 'nf4', # information theoretically optimal dtype for normally distributed weights
    bnb_4bit_use_double_quant = True, # quantize quantized weights //insert xzibit meme
    bnb_4bit_compute_dtype = torch.bfloat16 # optimized fp format for ML
)
model = make_model()
tokenizer = AutoTokenizer.from_pretrained(model_name)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.85k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/499M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/394 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/280 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [ ]:
print("Number of encoders/decoders:", model.config.num_hidden_layers)
print("Number of heads:", model.config.num_attention_heads)
# print(model.config)

Number of encoders/decoders: 12
Number of heads: 12
RobertaConfig {
  "_name_or_path": "cardiffnlp/tweet-topic-latest-multi",
  "architectures": [
    "RobertaForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "arts_&_culture",
    "1": "business_&_entrepreneurs",
    "2": "celebrity_&_pop_culture",
    "3": "diaries_&_daily_life",
    "4": "family",
    "5": "fashion_&_style",
    "6": "film_tv_&_video",
    "7": "fitness_&_health",
    "8": "food_&_dining",
    "9": "gaming",
    "10": "learning_&_educational",
    "11": "music",
    "12": "news_&_social_concern",
    "13": "other_hobbies",
    "14": "relationships",
    "15": "science_&_technology",
    "16": "sports",
    "17": "travel_&_adventure",
    "18": "youth_&_student_life"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072

In [ ]:
def find_size(model):
  total_size_in_bytes = sum(p.numel() * p.element_size() for p in model.parameters())
  total_size_in_megabytes = total_size_in_bytes / (1024 ** 2)
  print(f"Model size: {total_size_in_megabytes:.2f} MB")

find_size(model)

Model size: 475.54 MB


In [ ]:
# Load the GLUE dataset (SST-2 as an example)
dataset = load_dataset("cardiffnlp/tweet_topic_multi", split='validation_2020', trust_remote_code=True)
print("Sample from the dataset:", dataset[0])
# metric = evaluate.load("cardiffnlp/tweet_topic_multi", trust_remote_code=True)

tweet_topic_multi.py:   0%|          | 0.00/4.65k [00:00<?, ?B/s]

README.md:   0%|          | 0.00/9.23k [00:00<?, ?B/s]

Generating test_2020 split: 0 examples [00:00, ? examples/s]

Generating test_2021 split: 0 examples [00:00, ? examples/s]

Generating train_2020 split: 0 examples [00:00, ? examples/s]

...

Generating test_coling2022_random split: 0 examples [00:00, ? examples/s]

Generating train_coling2022_random split: 0 examples [00:00, ? examples/s]

Generating test_coling2022 split: 0 examples [00:00, ? examples/s]

Generating train_coling2022 split: 0 examples [00:00, ? examples/s]

Sample from the dataset: {'text': 'Major sidenote of the MSU game: How about the media team making custom graphics for the uniform!? Killing it!   Give em  a follow: {{USERNAME}} {{USERNAME}} {{USERNAME}} {{USERNAME}}', 'date': '2019-09-08', 'label': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0], 'label_name': ['sports'], 'id': '1170516407135522816'}


In [ ]:
print(len(dataset))
dataset[0:10]

573


{'text': ['Major sidenote of the MSU game: How about the media team making custom graphics for the uniform!? Killing it!   Give em  a follow: {{USERNAME}} {{USERNAME}} {{USERNAME}} {{USERNAME}}',
  'Another twinbill split for {@Down East Wood Ducks@} , setting up a decisive Game 5 against Fayetteville Sunday afternoon.  Winner advances, loser calls it a season.',
  'I’m just heart broken. A month ago I accepted we wore out on the finals, but here we are again last game at home and a terrible 2nd half has disappointed all the fans again. #NRLTigersSharks {{USERNAME}} ',
  'Did you know?   In 2007 when the Guardian reported on {@WikiLeaks@} first big leak, they only mentioned their source, #WikiLeaks, once - in the 12th paragraph!   Read more: chapter two of my book is now online! And free! #WLhistory {{URL}} ',
  'OvidiuZ94 {{USERNAME}} #Steam #game #giveaway in 119 minutes.Visit their #Twitch #stream for a chance to win a random game {{URL}}',
  '3-3 draw with {{USERNAME}} today in our

In [ ]:
# Preprocess the data
def preprocess_function(examples):
    tokenized_inputs = tokenizer(examples['text'], truncation=True, padding='max_length', max_length=128)
    tokenized_inputs['label'] = [sublist.index(1) for sublist in examples['label']]
    # tokenized_inputs['label'] = examples['label']
    return tokenized_inputs

encoded_dataset = dataset.map(preprocess_function, batched=True)

Map:   0%|          | 0/573 [00:00<?, ? examples/s]

In [ ]:
# Use DataLoader to create batches
encoded_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])
dataloader = DataLoader(encoded_dataset, batch_size=16)

In [ ]:
encoded_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])
pd.set_option('display.max_colwidth', None)
pd.set_option('display.expand_frame_repr', False)

dataloader = DataLoader(encoded_dataset, batch_size=16)


# Display a sample batch from the dataloader
for batch in dataloader:
    # print(batch['label'])
    input_ids = batch['input_ids'].tolist()
    attention_masks = batch['attention_mask'].tolist()
    labels = batch['label'].tolist()

    truncated_input_ids = [ids[:10] for ids in input_ids]  # Show only the first 10 tokens
    truncated_attention_masks = [masks[:10] for masks in attention_masks]

    df = pd.DataFrame({
        'Input IDs': truncated_input_ids,
        'Attention Mask': truncated_attention_masks,
        'Label': labels
    })

    print(df)
    break


                                                        Input IDs                  Attention Mask  Label
0              [0, 37038, 16742, 225, 6457, 9, 5, 6253, 791, 177]  [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]     16
1      [0, 21518, 9544, 24916, 3462, 13, 25522, 1039, 17853, 953]  [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]     16
2                    [0, 100, 17, 27, 119, 95, 1144, 3187, 4, 83]  [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]     16
3              [0, 20328, 47, 216, 116, 1437, 1437, 96, 3010, 77]  [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]     12
4   [0, 673, 21157, 9060, 1301, 6405, 47517, 47955, 48307, 46961]  [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]      9
5         [0, 246, 12, 246, 2451, 19, 47517, 47955, 48307, 46961]  [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]     16
6                [0, 771, 5361, 62, 42, 662, 7, 127, 25522, 1039]  [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]     16
7                [0, 18348, 261, 1020, 1547, 21, 421, 7, 310, 10]  [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]     16
8            [0, 21518, 849, 448, 25356, 177, 6, 277, 2

In [ ]:
# Modify model based on chromosome
def modify_model(model, chromosome):
    num_heads = model.config.num_attention_heads
    num_blocks = model.config.num_hidden_layers
    # num_ffn = model.config.num_hidden_layers

    # Disable attention heads
    heads_to_prune = defaultdict(list)
    for i, gene in enumerate(chromosome):
        if gene == 0:
          block_num = i//num_heads
          head_num = i%num_heads
          heads_to_prune[block_num].append(head_num)

    if heads_to_prune:

        for block_num in heads_to_prune:
              model.roberta.encoder.layer[block_num].attention.prune_heads(heads_to_prune[block_num])
    # return model

In [ ]:
def evaluate_fitness(chromosome, model, encoded_dataset):
    # Modify the model according to the chromosome
    if len(chromosome):
      model = make_model()
      modify_model(model, chromosome)
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)

    start_time = time.time()
    predictions = []
    labels = []

    with torch.no_grad():
        for batch in dataloader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            outputs = model(input_ids, attention_mask=attention_mask)

            # Get predictions
            logits = outputs.logits
            preds = torch.argmax(logits, dim=-1)
            predictions.extend(preds.cpu().numpy())
            labels.extend(batch['label'].cpu().numpy())


    # Evaluate the model
    # print("For the chromosome", chromosome)
    fitness = accuracy_score(labels, predictions)
    print(f"Validation Accuracy: {fitness:.4f}")
    end_time = time.time()
    print("Execution time = ",end_time-start_time)
    print()

    return fitness

In [ ]:
# class_mapping = model.config.id2label
# without modifying the model
print(evaluate_fitness([], model, encoded_dataset))

Validation Accuracy: 0.6178
Execution time =  4.624070167541504

0.6178010471204188


In [ ]:
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

Your runtime has 13.6 gigabytes of available RAM

Not using a high-RAM runtime


In [ ]:
def create_random_binary_list(length, percentage_of_zeros):
    # Calculate the number of zeros and ones based on the percentage
    num_zeros = int(length * (percentage_of_zeros / 100))
    num_ones = length - num_zeros

    # Create the list with the required number of 0s and 1s
    binary_list = [0] * num_zeros + [1] * num_ones

    # Shuffle the list to randomize the order
    random.shuffle(binary_list)

    return binary_list

def initialize_chromosome(num_genes):
  # initialize chromosome with given sparsity percentage
  return create_random_binary_list(num_genes, 30)

In [ ]:
# checking to see if decoder is there
print(model.config.is_decoder)

False


In [ ]:
model  = make_model()
print(find_size(model))
chromosome = initialize_chromosome(model.config.num_attention_heads*model.config.num_hidden_layers)
print("Chromosome:", chromosome)
fitness = evaluate_fitness(chromosome, model, encoded_dataset)
print("Fitness score (accuracy):", fitness)


Model size: 475.54 MB
None
Chromosome: [1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0]
12
defaultdict(<class 'list'>, {0: [5, 6, 10], 1: [4], 2: [0, 6, 8], 3: [1, 6, 7, 10], 4: [3, 7, 8], 5: [4, 6, 8, 10, 11], 6: [7, 8], 7: [1, 7, 10, 11], 8: [4, 6], 9: [0, 1, 2, 5, 9, 10], 10: [1, 2, 3, 4, 8], 11: [5, 7, 9, 10, 11]})
Model size: 443.26 MB
None
Validation Accuracy: 0.6143
Execution time =  3.480625629425049

Fitness score (accuracy): 0.6143106457242583


In [ ]:
# m  = AutoModelForSequenceClassification.from_pretrained(model_name, trust_remote_code=True, device_map=device)
# print(find_size(m), find_size(model))

In [ ]:
POPN_SIZE = 16
crossover_rate = 0.7
mutation_rate = 0.1

In [ ]:
def initialize_population(chromosome_length):
  # initialize random population
  population = []
  for _ in range(POPN_SIZE):
    chromosome = initialize_chromosome(chromosome_length)
    population.append(chromosome)
  return population

In [ ]:
# Selection (Tournament Selection)
'''
def select_parents(population, fitness_scores):
    parents = []
    for _ in range(POPN_SIZE):
        tournament = np.random.choice(POPN_SIZE, 2)
        print("tournament = ",tournament)
        winner = tournament[np.argmax(fitness_scores[tournament])]
        parents.append(population[winner])
    return np.array(parents)
    '''

'\ndef select_parents(population, fitness_scores):\n    parents = []\n    for _ in range(POPN_SIZE):\n        tournament = np.random.choice(POPN_SIZE, 2)\n        print("tournament = ",tournament)\n        winner = tournament[np.argmax(fitness_scores[tournament])]\n        parents.append(population[winner])\n    return np.array(parents)\n    '

In [ ]:
# roulette wheel selection
def select_parents(population, fitness_scores, num_parents):
    # Normalize fitness scores to create a probability distribution
    total_fitness = np.sum(fitness_scores)
    probabilities = fitness_scores / total_fitness

    # Select parents based on their fitness proportion (roulette wheel selection)
    # selected_parents = np.random.choice(population, size=num_parents, p=probabilities, replace=True)
    selected_parents = random.choices(population, weights=probabilities, k=num_parents)


    return np.array(selected_parents)

In [ ]:
# Crossover (Single-point crossover)
def crossover(parent1, parent2):
    if np.random.rand() < crossover_rate:
        point = np.random.randint(1, len(parent1) - 1)
        child1 = np.concatenate([parent1[:point], parent2[point:]])
        child2 = np.concatenate([parent2[:point], parent1[point:]])
    else:
        child1, child2 = parent1, parent2
    return child1, child2

In [ ]:
# Mutation (Flip bit mutation)
def mutate(chromosome):
    for i in range(len(chromosome)):
        if np.random.rand() < mutation_rate:
            chromosome[i] = 1 - chromosome[i]

    for c in range(model.config.num_attention_heads-1, len(chromosome), model.config.num_attention_heads):
      start = c-model.config.num_attention_heads-1
      enc_part = chromosome[start:c]
      num_ones = np.sum(enc_part)  # Count the number of 1s in the chromosome
      num_zeros = len(enc_part) - num_ones
      if num_ones==0:
        chromosome[start] = 1
    return chromosome

In [ ]:
def elitism_and_selection(population, fitness_scores, num_elites, num_parents):
    # Elitism: Keep the top num_elites individuals
    elite_indices = np.argsort(fitness_scores)[-num_elites:]  # Get indices of top individuals
    elites = [population[i] for i in elite_indices]

    # Perform roulette wheel selection for the rest of the parents
    remaining_population = np.delete(population, elite_indices, axis=0)
    remaining_fitness_scores = np.delete(fitness_scores, elite_indices)

    num_to_select = num_parents - num_elites
    selected_parents = select_parents(remaining_population, remaining_fitness_scores, num_to_select)

    # Combine elites and selected parents
    next_generation = np.vstack((elites, selected_parents))

    return next_generation

In [ ]:
'''def genetic_algorithm(model, num_generations, desired_sparsity):
  population = initialize_population(model.config.num_heads*model.config.num_layers)
  for generation in range(num_generations):
      fitness_scores = np.array([evaluate_fitness(chrom, model, encoded_dataset, metric) for chrom in population])
      parents = select_parents(population, fitness_scores)
      new_population = []
      for i in range(0, POPN_SIZE, 2):
          parent1, parent2 = parents[i], parents[i + 1]
          child1, child2 = crossover(parent1, parent2)
          child1 = mutate(child1)
          child2 = mutate(child2)
          new_population.extend([child1, child2])
      population = np.array(new_population)

      # Check for desired sparsity level
      sparsity_levels = np.mean(population == 0, axis=1)
      print(sparsity_levels)
      if np.any(sparsity_levels >= desired_sparsity):
          best_chromosome = population[np.argmax(sparsity_levels)]
          break
  return best_chromosome'''

'def genetic_algorithm(model, num_generations, desired_sparsity):\n  population = initialize_population(model.config.num_heads*model.config.num_layers)\n  for generation in range(num_generations):\n      fitness_scores = np.array([evaluate_fitness(chrom, model, encoded_dataset, metric) for chrom in population])\n      parents = select_parents(population, fitness_scores)\n      new_population = []\n      for i in range(0, POPN_SIZE, 2):\n          parent1, parent2 = parents[i], parents[i + 1]\n          child1, child2 = crossover(parent1, parent2)\n          child1 = mutate(child1)\n          child2 = mutate(child2)\n          new_population.extend([child1, child2])\n      population = np.array(new_population)\n\n      # Check for desired sparsity level\n      sparsity_levels = np.mean(population == 0, axis=1)\n      print(sparsity_levels)\n      if np.any(sparsity_levels >= desired_sparsity):\n          best_chromosome = population[np.argmax(sparsity_levels)]\n          break\n  return

In [ ]:
def genetic_algorithm(model, num_generations, desired_sparsity):
  population = initialize_population(model.config.num_attention_heads*model.config.num_hidden_layers)  # Initialize the population
  for generation in range(num_generations):
      fitness_scores = np.array([evaluate_fitness(chrom, model, encoded_dataset) for chrom in population])
      best_chromosome = population[np.argmax(fitness_scores)]
      print("new fitness scores:", fitness_scores)
      print(f"best chromosome in generation {generation} is {best_chromosome} with accuracy {fitness_scores[np.argmax(fitness_scores)]}")
      parents = elitism_and_selection(population, fitness_scores, 4, POPN_SIZE)
      # parents = select_parents(population, fitness_scores, POPN_SIZE)
      new_population = []
      for i in range(0, POPN_SIZE, 2):
          parent1, parent2 = parents[i], parents[i + 1]
          child1, child2 = crossover(parent1, parent2)
          child1 = mutate(child1)
          child2 = mutate(child2)
          new_population.extend([child1, child2])
      population = np.array(new_population)
  return best_chromosome

In [ ]:
'''genetic_algorithm(model, 10, 0.3)'''

'genetic_algorithm(model, 10, 0.3)'

In [ ]:
genetic_algorithm(model, 30, 0.3)
# print(model)

12
defaultdict(<class 'list'>, {0: [1, 4, 8], 1: [0, 2, 5, 6, 7, 8], 2: [4, 6, 8], 3: [1, 2, 9], 4: [0, 3, 5, 8, 9, 10], 5: [3, 5, 10], 6: [2, 4, 9], 7: [0, 4], 8: [10, 11], 9: [1, 3, 4, 7, 9], 10: [5, 7, 10], 11: [0, 3, 7, 11]})
Validation Accuracy: 0.6230
Execution time =  3.5612730979919434

12
defaultdict(<class 'list'>, {0: [1, 5, 6, 7, 9, 10], 1: [6, 7, 9, 11], 2: [1, 5, 6, 9], 3: [5, 9], 4: [2], 5: [4], 6: [0, 1, 5, 6, 7], 7: [0, 6, 9], 8: [0, 2, 4, 5, 7, 8], 9: [4, 5, 8], 10: [0, 1, 5, 8], 11: [4, 9, 10, 11]})
Validation Accuracy: 0.5881
Execution time =  3.6633033752441406

12
defaultdict(<class 'list'>, {0: [6, 11], 1: [3, 4, 10], 2: [2, 8], 3: [7, 8, 9, 10, 11], 4: [0, 1, 2, 3, 4, 5, 7], 5: [11], 6: [3, 4, 5, 9, 11], 7: [3, 6], 8: [2, 3, 10], 9: [1, 3, 4, 8, 9], 10: [1, 2, 4, 5, 9], 11: [2, 5, 11]})
Validation Accuracy: 0.6265
Execution time =  3.611550807952881

12
defaultdict(<class 'list'>, {0: [1, 5, 6, 9], 1: [1, 2, 5, 8, 9], 2: [6, 9], 4: [2, 9, 11], 5: [2, 4, 5, 6, 8,

array([1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1,
       1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0,
       1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0,
       0, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1,
       0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1,
       1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1,
       0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1])

In [ ]:
best_chromosome = [1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1]
evaluate_fitness(best_chromosome, model, encoded_dataset)

12
defaultdict(<class 'list'>, {0: [1, 4, 5, 6, 7, 8, 10], 1: [3, 6, 7, 11], 2: [2, 3, 5, 6, 7], 3: [0, 1, 5, 7], 4: [2, 5, 6, 9], 5: [2, 5, 6, 7, 9], 6: [2, 4, 5, 7, 8, 9], 7: [0, 2, 4, 5, 6, 8, 10, 11], 8: [0, 3, 4, 5, 8, 9], 9: [0, 3], 10: [0, 4, 5, 7, 9, 10], 11: [0, 1, 3, 6, 9]})
Validation Accuracy: 0.5969
Execution time =  3.377248525619507



0.5968586387434555

In [ ]:
model = make_model()
print("Original:")
find_size(model)
print("\nAfter modification:")
modify_model(model, best_chromosome)
find_size(model)

Original:
Model size: 475.54 MB

After modification:
Model size: 429.00 MB
